In [1]:
import os
import random as rand
import pandas as pd


path = os.getcwd()
path = os.chdir(os.path.normpath(os.getcwd() + os.sep + os.pardir))
path = os.getcwd()
os.chdir(path+"/Dataset")

ls = os.listdir()
newls = []
print(path)
print(ls)

/Users/cedricbeugre/Desktop/Deep-learning-Project-1
['.DS_Store', 'Test', 'Withoutmask', 'Withmask', 'Train', 'Incorrectly', 'Validation']


# Labeling Data

In [2]:
DB = []

print(os.getcwd())
os.chdir(os.getcwd()+"/Withoutmask")

dir = os.getcwd()
listdata = os.listdir()

for x in listdata:
    DB.append((dir+"/"+x,0))
    
os.chdir(os.path.normpath(os.getcwd() + os.sep + os.pardir))
os.chdir(os.getcwd()+"/WitHmask")

dir = os.getcwd()
listdata = os.listdir()

for x in listdata:
    DB.append((dir+"/"+x,1))

os.chdir(os.path.normpath(os.getcwd() + os.sep + os.pardir))
os.chdir(os.getcwd()+"/Incorrectly")

dir = os.getcwd()
listdata = os.listdir()

for x in listdata:
    DB.append((dir+"/"+x,2))




/Users/cedricbeugre/Desktop/Deep-learning-Project-1/Dataset


# Training/Test Dataset

In [3]:
rand.shuffle(DB)

X = []
Y = []

for x in DB:
    X.append(x[0])
    Y.append(x[1])

In [4]:
d = {'Y': Y, 'X': X}
df = pd.DataFrame(data=d)
df

,Y,X
0,2,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
1,2,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
2,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
3,1,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
4,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
...,...,...
15767,2,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
15768,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
15769,1,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
15770,1,/Users/cedricbeugre/Desktop/Deep-learning-Proj...


In [5]:
training_data = df.sample(frac=0.8, random_state=25)
testing_data = df.drop(training_data.index)

In [6]:
training_data

,Y,X
12583,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
11345,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
8898,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
14029,2,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
975,1,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
...,...,...
3179,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
11573,1,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
10577,2,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
14234,2,/Users/cedricbeugre/Desktop/Deep-learning-Proj...


In [7]:
testing_data

,Y,X
8,1,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
14,2,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
23,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
25,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
26,2,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
...,...,...
15739,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
15752,2,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
15755,0,/Users/cedricbeugre/Desktop/Deep-learning-Proj...
15758,1,/Users/cedricbeugre/Desktop/Deep-learning-Proj...


In [8]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt"

while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
    h, w = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces = net.forward()

    for i in range(0, faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            print(box)
            (x, y, x1, y1) = box.astype("int")
            cv2.rectangle(frame, (x, y), (x1, y1), (0, 0, 255), 3)
 
    cv2.imshow('fe',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

 

error: OpenCV(4.6.0) /Users/runner/miniforge3/conda-bld/libopencv_1666820195419/work/modules/dnn/src/caffe/caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "deploy.prototxt" in function 'ReadProtoFromTextFile'
